<a href="https://colab.research.google.com/github/saulobritto/bioinfo-tools-gcolab/blob/main/Supplementary_Material.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---


#**Whole genome sequencing of 53 Brucella abortus strains reveals variants possible associated with reduced susceptibility to rifampicin and resistance to multiple antimicrobial agents**


---



##Code for PARSNP, .gbk PROKKA file and CARD Intersection

    This code gets in .gbk PROKKA files the genes that match 
    the locus_tag gave by PARSNP run and intersect this 
    search with the CARD database genes list.

---

    Author:
      Saulo Britto da Silva
        saulobdasilva@gmail.com
        https://github.com/saulobritto

    > Last update: 08 Jan. 2021

---
    [BIOPYTHON]
    Peter J. A. Cock, Tiago Antao, Jeffrey T. Chang, Brad A. Chapman, Cymon J. Cox, Andrew Dalke, Iddo Friedberg, Thomas Hamelryck, Frank Kauff, Bartek Wilczynski, Michiel J. L. de Hoon:
    Biopython: freely available Python tools for computational molecular biology and bioinformatics. 
    Bioinformatics, Volume 25, Issue 11, 1 June 2009, Pages 1422–1423, https://doi.org/10.1093/bioinformatics/btp163


In [ ]:
#biopython install
!pip install biopython

In [ ]:
#libraries import
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO
from io import StringIO
import pathlib
import pandas as pd
import numpy as np
import json
from itertools import chain

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#locus_tags file import, organized in excel with the cro1 + cro2 variants generated by parsnp
from google.colab import files
files.upload()

In [ ]:
#DataFrame created from file
df_locus_tags = pd.read_csv('/file-imported-path', ',')

#create the locus_tag list
locus_tag_list = df_locus_tags['column-name-with-locus_tags'].tolist()

print(df_locus_tags)
print(locus_tag_list)

In [ ]:
#path to gbk file folders of both chromosomes generated by PROKKA for the Brucella abortus ref 2308
ref1 = list(pathlib.Path("/content/drive/MyDrive/IC UFLA/Referências/prokka_2308_cro1").glob("*.gbk"))
ref2 = list(pathlib.Path("/content/drive/MyDrive/IC UFLA/Referências/prokka_2308_cro2").glob("*.gbk"))

#join path
ref = ref1 + ref2

genes = []

#search from the PARSNP locus_tag, the genes an products from the reference
for file in ref:
  recs = SeqIO.to_dict(SeqIO.parse(file, "genbank"))
  for i in recs:
    for f in recs[i].features:
        if f.type == "CDS":
           for z in locus_tag_list:
            if z in f.qualifiers.get('locus_tag', []):
                print(f)
                #create the list with genes and proteins
                itens = [f.qualifiers.get('gene', [])]
                genes.extend(itens)

#print the list, for checking results                
print(genes)

In [ ]:
#genes found DataFrame
g_p_df = pandas.DataFrame(genes)
g_p_df.columns =['Gene']
print(g_p_df)

#create a list with the genes
genes_match = g_p_df['Gene'].tolist()

print('\n', 'Genes match from the intersection between PROKKA and PARSNP:', '\n', genes_match)

In [ ]:
#corrects the result above, splitting genes followed by a "_1 or _2", getting only the gene name
genes_prokka_parsnp = []
for i in genes_match:
    z = str(i).split('_')
    genes_prokka_parsnp.append(z)

#corrects the result above, that found lists inside a list
correction_list = []
for l in genes_prokka_parsnp:
  for i in l:
    correction_list.append(i)

genes_prokka_parsnp = correction_list

print('The genes found by the intersection between PROKKA and PARSNP: ', '\n', genes_prokka_parsnp)

In [ ]:
#upload the 'card.JSON' file version 3.1.0 from CARD Database 
files.upload()

In [ ]:
#import and read JSON file
file_name = '/content/card.json'
with open(file_name, 'r') as f:
    document =  json.loads(f.read())

#create a DataFrame
card_json = pandas.DataFrame.from_dict(document, orient='columns')
print(card_json.head())

#select the genes row itens
genes_row = card_json.iloc[1].tolist()

#some results were phrases with genes, this code below splits the phrases and we get the genes separated
genes_row_corrected = []
for i in genes_row:
  z = str(i).split(' ')
  genes_row_corrected.append(z)

#normaliza os dados que ficaram em listas dentro de listas para uma lista única, para realizar a interseção
genes_card = []
for l in genes_row_corrected:
  for i in l:
    genes_card.append(i)
print('\n', 'This is the genes list from CARD Database:', '\n', genes_card)

In [ ]:
total_intersection = list(set(genes_prokka_parsnp) & set(genes_card))
print('These are the final genes from the intersection between PROKKA, PARSNP and CARD Database:', '\n', total_intersection)